In [1]:
import numpy as np
import pandas as pd
import datetime
import random

In [2]:
nusers=1000

In [3]:
sessions=np.random.poisson(lam=1.5,size=nusers)

In [4]:
df=pd.DataFrame({'userId':[],'sessionId':[],'eventTime':[],'pageName':[]})

In [5]:
def nextpage(x):

    if x=='start':
        options=['question','info','step1','exit']
        prob=[0.1,0.3,0.3,0.3]
    elif x=='question':
        options=['start','exit']
        prob=[0.8,0.2]
    elif x=='info':
        options=['start','exit']
        prob=[0.7,0.3]
    elif x=='step1':
        options=['help1','step2','exit','start']
        prob=[0.2,0.6,0.1,0.1]
    elif x=='help1':
        options=['step1','exit']
        prob=[0.9,0.1]
    elif x=='step2':
        options=['help2','checkout','exit','step1']
        prob=[0.5,0.3,0.1,0.1]
    elif x=='help2':
        options=['step2','exit']
        prob=[0.6,0.4]
    elif x=='checkout':
        options=['step2','success','exit','error']
        prob=[0.1,0.1,0.5,0.3]
    elif x=='error':
        options=['checkout','exit']
        prob=[0.2,0.8]
    return random.choices(options,weights=prob,k=1)[0]

In [6]:
for i in range(nusers):
    start=datetime.date(2025,1,1)
    end=datetime.date(2025,2,1)
    for j in range(sessions[i]):
        daysd=(end-start).days
        daysr=start+datetime.timedelta(days=random.randrange(daysd))
        hoursr=random.randint(0,23)
        minutesr=random.randint(0,59)
        secondsr=random.randint(0,59)
        timer=datetime.datetime(daysr.year,daysr.month,daysr.day,hoursr,minutesr,secondsr)
        df.loc[len(df)]=[f'user_{i}',f'u{i}s{j}',timer,'start']
        while df['pageName'][len(df)-1]!='exit' and df['pageName'][len(df)-1]!='success':
            minutesr=random.randint(0,9)
            secondsr=random.randint(0,59)
            timer+=datetime.timedelta(minutes=minutesr,seconds=secondsr)
            df.loc[len(df)]=[f'user_{i}',f'u{i}s{j}',timer,nextpage(df['pageName'][len(df)-1])]
        start=datetime.date(timer.year,timer.month,timer.day)+datetime.timedelta(days=1)
        end+=datetime.timedelta(days=2)

In [7]:
df=df.sort_values(by='eventTime').reset_index(drop=True)

In [8]:
indexes=df.sample(int(len(df)*0.01)).index
dup=df.loc[indexes].copy()
df1=pd.concat([df,dup],ignore_index=True).reset_index(drop=True)

indexes=df1.sample(int(len(df1)*0.01)).index
dup=df1.loc[indexes].copy()
diff=np.random.randint(-5,6,size=len(dup))
dup['eventTime']=pd.to_datetime(dup['eventTime'])+pd.to_timedelta(diff,unit='s')
df2=pd.concat([df1,dup],ignore_index=True).reset_index(drop=True)

indexes=df2.sample(int(len(df2)*0.01)).index
df3=df2.copy()
df3.loc[indexes,'pageName']=np.nan

df4=df3.copy()
indexes=df4[df4['pageName'].notna()].sample(int(len(df4)*0.01)).index
df4.loc[indexes,'pageName']=df4.loc[indexes,'pageName'].str.title()

df5=df4.copy()
indexes=df5.sample(int(len(df5)*0.001)).index
df5.loc[indexes,'eventTime']=pd.to_datetime('1970-01-01')

df6=df5.copy()
df6['device']=random.choices(['IOS','Android','Web'],[0.4,0.4,0.2],k=len(df6))

In [9]:
df6.to_csv('simulatedData.csv',index=False)